In [2]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score


In [3]:
# Load the data credit card dataset
data = pd.read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
#give a dataframe od data with class 0
data_0 = data[data['Class']==0]
#give a dataframe od data with class 1
data_1 = data[data['Class']==1]

In [6]:
data_1

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [82]:
#take 492 class=0 data and join it with all class =1 data and shuffle the data
data0 = data[data['Class'] == 0].sample(492)
data0
#take 492 class=1 data
data1 = data[data['Class'] == 1]
#data0+data1
data = pd.concat([data0, data1])
data        
#shuffle randomly 'data dataframe'
data = data.sample(frac=1).reset_index(drop=True)
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,35585.0,-2.019001,1.491270,0.005222,0.817253,0.973252,-0.639268,-0.974073,-3.146929,-0.003159,...,2.839596,-1.185443,-0.142812,-0.086103,-0.329113,0.523601,0.626283,0.152440,0.76,1
1,121630.0,-1.015058,0.864680,-0.804007,-0.766888,-0.285634,-0.198558,2.228691,-0.086907,-0.469054,...,0.218353,0.559478,-0.138572,0.686360,0.349222,-0.597351,0.010021,-0.018155,297.79,0
2,27252.0,-25.942434,14.601998,-27.368650,6.378395,-19.104033,-4.684806,-18.261393,17.052566,-3.742605,...,1.784316,-1.917759,-1.235787,0.161105,1.820378,-0.219359,1.388786,0.406810,99.99,1
3,54009.0,-9.059783,-5.766039,-2.120070,0.282604,-2.504822,0.585238,0.311152,1.127436,-0.053737,...,-0.339397,0.657815,-1.548497,0.017892,0.607426,-0.186774,-0.880670,-0.593960,239.76,0
4,36110.0,1.268610,0.069977,0.331383,0.725081,-0.315020,-0.434386,-0.032536,-0.105304,0.499264,...,-0.265292,-0.576113,-0.048183,-0.082980,0.520402,0.300388,-0.022789,0.007630,5.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,135095.0,0.232512,0.938944,-4.647780,3.079844,-1.902655,-1.041408,-1.020407,0.547069,-1.105990,...,0.911373,1.042929,0.999394,0.901260,-0.452093,0.192959,0.180859,-0.029315,345.00,1
980,161703.0,-0.575111,0.004303,-1.419063,-1.349239,1.895995,-1.237986,1.965465,-0.981780,0.268958,...,0.191535,1.443727,0.529104,0.750045,-1.381799,-0.317804,-0.295994,-0.292327,42.05,0
981,34095.0,0.983613,-0.603226,1.159121,-0.220913,-0.571505,1.469845,-1.154125,0.629490,0.864138,...,0.336173,1.113044,0.094973,-0.857703,-0.254082,1.269280,0.036998,0.012477,36.97,0
982,129482.0,-0.188626,0.836603,-0.121941,0.918496,0.970639,-0.480418,1.844671,-0.464155,-0.394837,...,0.084726,0.541758,-0.146089,-0.434450,0.065981,-0.378582,0.237106,0.022007,100.48,0


In [83]:
# Extract features and labels
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values


In [84]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [86]:
# Quantum feature map
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

In [87]:
@qml.qnode(dev)
def quantum_circuit(params, x):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.RZ(params[0], wires=0)
    qml.RZ(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=1)  # Use the last parameter for RY on the second qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [88]:
# Initialize the quantum circuit parameters
params = np.random.uniform(low=0, high=2 * np.pi, size=(3,))

In [89]:
# Evaluate the quantum circuit for the training data
feature_map_train = np.array([quantum_circuit(params, x) for x in X_train])

In [90]:
# Use the quantum feature map to transform the testing data
feature_map_test = np.array([quantum_circuit(params, x) for x in X_test])

In [91]:

svm = SVC(kernel='precomputed')
gram_matrix_train = np.dot(feature_map_train, feature_map_train.T)
svm.fit(gram_matrix_train, y_train)

SVC(kernel='precomputed')

In [92]:
# Predict on the testing data
gram_matrix_test = np.dot(feature_map_test, feature_map_train.T)
y_pred = svm.predict(gram_matrix_test)


In [94]:
# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy: {accuracy}")
#evaluate f1 score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

Classification Accuracy: 0.4720812182741117
F1 Score: 0.6413793103448276
